In [48]:
import altair as alt
import pandas as pd
df = pd.read_csv("base de datos turismo - copia.csv", sep=";")
df = df[~df["Nacionalidad"].str.contains("TOTAL", case=False, na=False)]
df.head(5)

df_largo = df.melt(
    id_vars=["Nacionalidad", "Continente"],
    var_name="Mes",
    value_name="Turistas")
df_largo["Turistas"] = df_largo["Turistas"].replace({"\.": "", ",": ""}, regex=True)
df_largo["Turistas"] = pd.to_numeric(df_largo["Turistas"], errors="coerce").fillna(0)

meses_es = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr", "may": "May",
    "jun": "Jun", "jul": "Jul", "ago": "Aug", "sept": "Sep", "oct": "Oct",
    "nov": "Nov", "dic": "Dec"
}
for esp, eng in meses_es.items():
    df_largo["Mes"] = df_largo["Mes"].str.replace(f"^{esp}", eng, regex=True)
    df_largo["Fecha"] = pd.to_datetime(df_largo["Mes"], format="%b-%y", errors="coerce")
df_largo = df_largo.dropna(subset=["Fecha"])
df_largo["Mes_Año"] = df_largo["Fecha"].dt.to_period("M").astype(str)
meses_es_inv = {
    1: "ene", 2: "feb", 3: "mar", 4: "abr", 5: "may", 6: "jun",
    7: "jul", 8: "ago", 9: "sept", 10: "oct", 11: "nov", 12: "dic"
}
df_largo["Etiqueta_esp"] = df_largo["Fecha"].dt.month.map(meses_es_inv) + " " + df_largo["Fecha"].dt.year.astype(str)


paises_objetivo = ["ARGENTINA", "BRASIL", "PERU", "BOLIVIA", "ESTADOS UNIDOS", "COLOMBIA"]
df_filtrado = df_largo[df_largo["Nacionalidad"].isin(paises_objetivo)]
df_filtrado = df_filtrado[df_filtrado["Fecha"].dt.year < 2025]

df_filtrado = df_filtrado[df_filtrado["Fecha"].dt.month.isin([1, 4, 7, 10])]


label_map_js = {...}
label_map_js = {row["Mes_Año"]: row["Etiqueta_esp"] for _, row in meses_label.iterrows()}

label_expr = "{" + ", ".join([f'\"{k}\": \"{v}\"' for k, v in label_map_js.items()]) + "}[datum.value]"

chart = alt.Chart(df_filtrado).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X(
        "Mes_Año:O",
        title="Mes",
        axis=alt.Axis(
            labelAngle=-45,
            labelFontSize=8,
            labelExpr=label_expr
        )
    ),
    y=alt.Y("sum(Turistas):Q", title="Total de turistas"),
    color=alt.Color("Nacionalidad:N", title="País"),
    tooltip=["Mes_Año", "Nacionalidad", "sum(Turistas)"]
).properties(
    width=700,
    height=400,
    title="Ingreso de turistas por país y mes (2009-2024)"
).interactive()
chart


alt.Chart(...)